Outline

[Preliminary](#preliminary)<br>
[Clean & Merge](#clean--merge)<br>
[Feature engineering](#feature-engineering)<br>
[Graphical Analysis](#graphical-analysis)<br>
[Modelling](#modelling)<br>


# Preliminary

In [34]:
import glob
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dateutil import tz

data_directory = '../../RawFiles'
melbourne_tz = tz.gettz('Australia/Melbourne')

sns.set()

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x if int(x) != x else '%d' % int(x))


class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)



<a id='Clean_Merge'></a>

# Clean & Merge

channel_cd	voltage_lvt	amps	power_factor_pf	number_of_phases	overground_or_underground	make	model	charger	drive_kms	has_ev

In [2]:
def append_csvs_into_df(file_pattern: str, ignore_index: bool = True, sort: bool = False):
    data_frames = []
    for file in glob.glob(file_pattern):
        print(f'loading: {file}')
        data_frames.append(pd.read_csv(file))
    full_dataset = pd.concat(data_frames, ignore_index=ignore_index, sort=sort)
    return full_dataset


def simple_col_name(x):
    x = x.lower().strip()
    return {'meter_id': 'meter',
            'start_timestamptz_ts': 'time',
            'interval_ts': 'time',
            'active_consumption_kwh': 'consumption',
            'active_generation_kwh': 'generation',
            'ev make': 'make',
            'ev model': 'model',
            "amps_lct": 'amps',
            'ev wall charger installed?': 'charger',
            'how many kms do you usually drive each year? \n(prior to the covid-19 pandemic)': 'drive_kms',
            'power_factor_pf': 'power factor',
            ''
            }.setdefault(x, x)



<h1> 1- EV Meters list (those who have EV) </h1>

In [3]:
df_with_ev = pd.read_excel(f'{data_directory}/EV_training_meters_list.xlsx').rename(columns=simple_col_name).drop(columns=['kms_group'])
meters_with_ev = df_with_ev['meter'].to_list()
df_with_ev['has_ev'] = 1
df_with_ev.head()


,meter,make,model,charger,drive_kms,has_ev
0,14702,Tesla,Model 3 and Model X,Yes,"Greater than 20,000",1
1,21463,Nissan,Leaf,No,"5,000 to 10,000",1
2,63317,Nissan,Leaf,No,"5,000 to 10,000",1
3,69825,Jaguar,ipace,Yes,"10,000 to 15,000",1
4,98536,Jaguar,ipace,No,"15,000 to 20,000",1



<h1> 2- Consumption datasets </h1>


In [4]:
# append & rename
df_consumption = append_csvs_into_df(f"{data_directory}/*consumption*nov*21*csv").drop(['Unnamed: 0'], axis=1).rename(
    columns=simple_col_name)
# dates
df_consumption['time'] = pd.to_datetime(df_consumption['time'], infer_datetime_format=True, utc=True).dt.tz_convert(
    melbourne_tz)
# fixing columns
df_consumption = df_consumption.groupby(['meter', 'time']).sum().reset_index()
df_consumption['total_consumption'] = df_consumption[['consumption', 'generation']].sum(axis=1, skipna=True)
# merge with known EVs
df_consumption = pd.merge(left=df_consumption, right=df_with_ev, on='meter', how='outer')
# has_ev
df_consumption['has_ev'] = df_consumption['has_ev'].fillna(0).astype('int')
# needed later
meters_with_no_ev = list(set(df_consumption['meter'].unique().tolist()) - set(meters_with_ev))
# index
df_consumption = df_consumption.set_index(['meter', 'time']).sort_index()
# .drop_duplicates(subset=['meter', 'time'], keep=False).drop('element_uuid', axis=1)\
# .set_index(['time'])\
# .groupby('meter')\
# .resample('5T', closed='right', label='right')\
# .ffill().drop('meter', axis=1)
df_consumption.head()



loading: ../../RawFiles\consumption_training_data_nov21.csv


consumption  generation  total_consumption  \
meter time                                                                    
396   2021-11-01 11:00:00+11:00       0.0437         0.0             0.0437   
      2021-11-01 11:30:00+11:00       0.0750         0.0             0.0750   
      2021-11-01 12:00:00+11:00       0.1187         0.0             0.1187   
      2021-11-01 12:30:00+11:00       0.0562         0.0             0.0562   
      2021-11-01 13:00:00+11:00       0.0437         0.0             0.0437   

                                make model charger drive_kms  has_ev  
meter time                                                            
396   2021-11-01 11:00:00+11:00  NaN   NaN     NaN       NaN       0  
      2021-11-01 11:30:00+11:00  NaN   NaN     NaN       NaN       0  
      2021-11-01 12:00:00+11:00  NaN   NaN     NaN       NaN       0  
      2021-11-01 12:30:00+11:00  NaN   NaN     NaN       NaN       0  
      2021-11-01 13:00:00+11:00  NaN   NaN     NaN       NaN       0

<h1> 3- Power quality datasets </h1>

In [5]:
# power quality datasets
# append & rename
df_power_quality = append_csvs_into_df(f"{data_directory}/*power_quality*nov*21*csv").drop(['Unnamed: 0'],
                                                                                           axis=1).rename(
    columns=simple_col_name)
# proper types
df_power_quality['time'] = pd.to_datetime(df_power_quality['time'], infer_datetime_format=True, utc=True).dt.tz_convert(
    melbourne_tz)
obj_columns = df_power_quality.select_dtypes(include='object').columns
df_power_quality[obj_columns] = df_power_quality[obj_columns].astype('category')
# merge with known EVs
df_power_quality = pd.merge(left=df_power_quality, right=df_with_ev, on='meter', how='outer')
df_power_quality['has_ev'] = df_power_quality['has_ev'].fillna(0).astype('int')
# index
df_power_quality = df_power_quality.set_index(['meter', 'time']).sort_index()
df_power_quality.head()


loading: ../../RawFiles\power_quality_training_data_nov21.csv


channel_cd  voltage_lvt      amps  \
meter time                                                          
396   2021-11-01 00:00:00+11:00          a       242.47  0.344000   
      2021-11-01 00:05:00+11:00          a       244.95  0.335175   
      2021-11-01 00:10:00+11:00          a       243.59  0.331522   
      2021-11-01 00:15:00+11:00          a       244.58  0.566290   
      2021-11-01 00:20:00+11:00          a       245.43  0.540289   

                                 power_factor_pf  number_of_phases  \
meter time                                                           
396   2021-11-01 00:00:00+11:00            1.000                 1   
      2021-11-01 00:05:00+11:00           -0.543                 1   
      2021-11-01 00:10:00+11:00           -0.552                 1   
      2021-11-01 00:15:00+11:00           -0.973                 1   
      2021-11-01 00:20:00+11:00           -0.968                 1   

                                overground_or_underground make model charger  \
meter time                                                                     
396   2021-11-01 00:00:00+11:00                        OH  NaN   NaN     NaN   
      2021-11-01 00:05:00+11:00                        OH  NaN   NaN     NaN   
      2021-11-01 00:10:00+11:00                        OH  NaN   NaN     NaN   
      2021-11-01 00:15:00+11:00                        OH  NaN   NaN     NaN   
      2021-11-01 00:20:00+11:00                        OH  NaN   NaN     NaN   

                                drive_kms  has_ev  
meter time                                         
396   2021-11-01 00:00:00+11:00       NaN       0  
      2021-11-01 00:05:00+11:00       NaN       0  
      2021-11-01 00:10:00+11:00       NaN       0  
      2021-11-01 00:15:00+11:00       NaN       0  
      2021-11-01 00:20:00+11:00       NaN       0

# Feature engineering

The Correlation between the number of times within a month the meter consumption exceeded different percentiles of consumption and probability of having EV

In [ ]:
corr = []
perc = []
for q in range(50, 100, 5):
    df_consumption[f'cons_above_{q}'] = np.where(
        df_consumption['consumption'] > df_consumption['consumption'].quantile(q/100), 1, 0
        )
    aggregations = {
        f'cons_above_{q}': 'sum',
        'has_ev': 'last'
    }
    numeric_features = df_consumption.select_dtypes(exclude='object').columns
    agg_cons = df_consumption[numeric_features].reset_index().groupby('meter').aggregate(aggregations)
    corr.append(agg_cons.corr().loc['has_ev', f'cons_above_{q}'])
    perc.append(q)
pd.Series(corr, index=perc)
df_consumption.head()

In [ ]:
pd.DataFrame([df_power_quality.isna().sum(), df_power_quality.count()], index=['count_missing', 'count']).T

The Correlation between the number of times within a month the meter current (amps) exceeded different percentiles of current and probability of having EV

In [ ]:
corr = []
perc = []
for q in range(50, 100, 5):
    df_power_quality[f'amps_above_{q}'] = np.where(
        df_power_quality['amps'] > df_power_quality['amps'].quantile(q/100), 1, 0
        )
    aggregations = {
        f'amps_above_{q}': 'sum',
        'has_ev': 'last'
    }
    numeric_features = df_power_quality.select_dtypes(exclude='object').columns
    agg_amps = df_power_quality[numeric_features].reset_index().groupby('meter').aggregate(aggregations)
    corr.append(agg_amps.corr().loc['has_ev', f'amps_above_{q}'])
    perc.append(q)
pd.Series(corr, index=perc)

# Graphical Analysis

Helper functions

In [ ]:

def subset_by_meter(meter_id: int, start: str, end: str, cols=None):
    # ensure start and end dates are within dataset range
    cols = cols or ['consumption', 'amps']
    min_date, max_date = df_consumption.reset_index()['time'][[0, len(df_consumption) - 1]]
    start = max(pd.to_datetime(start).tz_localize(melbourne_tz), min_date)
    end = min(pd.to_datetime(end).tz_localize(melbourne_tz), max_date)
    df1 = df_consumption.loc[pd.IndexSlice[meter_id, start:end], :].droplevel('meter')
    df2 = df_power_quality.loc[pd.IndexSlice[meter_id, start:end], :].droplevel('meter')
    return pd.merge(left=df1, right=df2, left_index=True, right_index=True, how='left')[cols]


def plot_subplots_common_x(meter_data, hours_interval: int | None = None) -> None:
    fig, axes = plt.subplots(2, 1, figsize=(30, 12))
    # plot at a time
    for col, ax in zip(meter_data.columns, axes):
        meter_data[col].plot(ax=ax, subplots=True, x_compat=True)
        ax.set_title(col.title())
        ax.set_xlabel("")
        # X axis labels
        date_locator = mdates.DayLocator(interval=1)
        date_form = mdates.DateFormatter("%m-%d")
        ax.xaxis.grid(True, which='major', color='gray')
        ax.xaxis.set_major_locator(date_locator)
        ax.xaxis.set_major_formatter(date_form)
        ax.tick_params(axis='x', which='major', pad=20)
        if hours_interval is not None:
            hour_locator = mdates.HourLocator(interval=hours_interval)
            hour_form = mdates.DateFormatter("%H")
            ax.xaxis.set_minor_locator(hour_locator)
            ax.xaxis.set_minor_formatter(hour_form)

def plot_consumption_amps(meter, start='2021-11-15', end='2021-11-30', hours_interval=12, verbose=True):
    meter_data = subset_by_meter(meter_id=meter, start=start, end=end)
    if verbose:
        meter_info = df_with_ev[df_with_ev['meter'] == meter]
        if not meter_info.empty:
            print(meter_info)
        else:
            print('meter has no EV')
    
    fig = plot_subplots_common_x(meter_data, hours_interval=hours_interval)


Consumption and current for meters with EV between 15th and 30th November 2021

In [ ]:
plot_consumption_amps(meter=meters_with_ev[0])
## example of customized plot
plot_consumption_amps(meter=meters_with_ev[0], start='2021-11-15', end='2021-11-17', hours_interval=6, verbose=False)

In [ ]:
plot_consumption_amps(meter=meters_with_ev[1])

In [ ]:
plot_consumption_amps(meters_with_ev[2])
plot_consumption_amps(meters_with_ev[2], start='2021-11-23', end='2021-11-26', verbose=False)

In [ ]:

plot_consumption_amps(meters_with_ev[3])

In [ ]:
plot_consumption_amps(meters_with_ev[4])

In [ ]:
plot_consumption_amps(meters_with_ev[5])

In [ ]:
plot_consumption_amps(meters_with_ev[6])

In [ ]:
plot_consumption_amps(meters_with_ev[7])

In [ ]:
plot_consumption_amps(meters_with_ev[8])

In [ ]:
plot_consumption_amps(meters_with_ev[9])

In [ ]:
plot_consumption_amps(meters_with_ev[10])

In [ ]:
plot_consumption_amps(meters_with_ev[11])

In [ ]:
plot_consumption_amps(meters_with_ev[12])

In [ ]:
plot_consumption_amps(meters_with_ev[13])

In [ ]:
plot_consumption_amps(meters_with_ev[14])

In [ ]:
plot_consumption_amps(meters_with_ev[15])


<h3> No EV meters for same period as above </h3>

In [ ]:
plot_consumption_amps(meters_with_no_ev[0])
plot_consumption_amps(meters_with_no_ev[0], start='2021-11-15', end='2021-11-19', verbose=False)

In [ ]:
plot_consumption_amps(meters_with_no_ev[1])

In [ ]:
plot_consumption_amps(meters_with_no_ev[2])

In [ ]:
plot_consumption_amps(meters_with_no_ev[3])

In [ ]:
plot_consumption_amps(meters_with_no_ev[4])

In [ ]:
plot_consumption_amps(meters_with_no_ev[5])

In [ ]:

plot_consumption_amps(meters_with_no_ev[7])

In [ ]:
plot_consumption_amps(meters_with_no_ev[8])

In [ ]:
plot_consumption_amps(meters_with_no_ev[9])


In [ ]:
plot_consumption_amps(meters_with_no_ev[10])

In [ ]:
plot_consumption_amps(meters_with_no_ev[11])

In [ ]:
plot_consumption_amps(meters_with_no_ev[12])

In [ ]:
plot_consumption_amps(meters_with_no_ev[13])
plot_consumption_amps(meters_with_no_ev[13], start='2021-11-20', end='2021-11-24', hours_interval=3)

In [ ]:
plot_consumption_amps(meters_with_no_ev[14])

In [ ]:
plot_consumption_amps(meters_with_no_ev[15])

<a id='Modelling'></a>

# Modelling

### old approach 

In [ ]:
df_consumption['has_ev'] = np.where(df_consumption['has_ev'].isna(), 0, 1)
# for col in ['year', 'month', 'day', 'hour', 'minute']:
#     df_consumption[col] = getattr(pd.DatetimeIndex(df_consumption['time']), col)
df_consumption_analysis = df_consumption.drop(
    columns=['generation', 'ev_make', 'ev_model', 'wallcharger', 'annual_km', 'annual_km_group',
             'year', 'month', 'day', 'hour', 'minute'])
df_consumption_analysis['identifier'] = df_consumption_analysis['meter'].astype('str') + df_consumption_analysis[
    'element_uuid']
df_consumption_analysis = df_consumption_analysis.drop(columns=['meter', 'element_uuid']).set_index(
    ['identifier', 'time'])
df_consumption_analysis.head()

In [ ]:
import statsmodels.api as sm
from linearmodels import PanelOLS

exog = sm.tools.add_constant(df_consumption_analysis['consumption'])
endog = df_consumption_analysis['has_ev']
# fixed effects model
model_fe = PanelOLS(endog, exog, entity_effects=True)
fe_res = model_fe.fit()
print(fe_res)

In [ ]:
df_power_quality['has_ev'] = np.where(df_power_quality['has_ev'].isna(), 0, 1)
# for col in ['year', 'month', 'day', 'hour', 'minute']:
#     df_power_quality[col] = getattr(pd.DatetimeIndex(df_power_quality['time']), col)
df_power_quality.drop(
    columns=['overground_or_underground', 'ev_make', 'ev_model', 'wallcharger', 'annual_km', 'annual_km_group'],
    inplace=True)
df_power_quality['identifier'] = df_power_quality['meter'].astype('str') + df_power_quality['channel_cd']
df_power_quality.drop(columns=['meter', 'channel_cs'], inpalce=True)
df_power_quality.set_index(['identifier', 'time'], inplace=True)
df_power_quality.head()

In [ ]:
# import statsmodels.api as sm
# from linearmodels import PanelOLS

# exog = sm.tools.add_constant(df_power_quality['amps'])
# exog = sm.tools.add_constant(df_power_quality['voltage_lvt'])
# exog = sm.tools.add_constant(df_power_quality['power_factor_pf'])
# exog = sm.tools.add_constant(df_power_quality['number_of_phases'])
# endog = df_consumption_analysis['has_ev']
# # fixed effects model
# model_fe = PanelOLS(endog, exog, entity_effects=True)
# fe_res = model_fe.fit()
# print(fe_res)

## Approach one

- Collapsing channels<br> 
    phase 1: drop channel b<br>
    phase 3: collapse channels; voltage -> mean, amps -> sum, power factor -> mean<br>
- Collapsing variables to be weekly averages (as features)<br>
- Logistic regression<br>

(note: element_uuid is dropped from consumption)

In [39]:
df_consumption_app01 = df_consumption.copy()
df_consumption_app01.info()
display('df_consumption_app01.describe().T', f'df_consumption_app01.head()')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1537672 entries, (396, Timestamp('2021-11-01 11:00:00+1100', tz='dateutil/Australia/Melbourne')) to (792189, Timestamp('2021-12-01 10:30:00+1100', tz='dateutil/Australia/Melbourne'))
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   consumption        1537672 non-null  float64
 1   generation         1537672 non-null  float64
 2   total_consumption  1537672 non-null  float64
 3   make               85909 non-null    object 
 4   model              85909 non-null    object 
 5   charger            85909 non-null    object 
 6   drive_kms          85909 non-null    object 
 7   has_ev             1537672 non-null  int32  
dtypes: float64(3), int32(1), object(4)
memory usage: 94.2+ MB


df_consumption_app01.describe().T
                    count    mean     std  min     25%     50%     75%  \
consumption       1537672 0.19558 0.35758    0 0.03120 0.08120 0.20000   
generation        1537672 0.04047 0.24043    0       0       0       0   
total_consumption 1537672 0.23605 0.41330    0 0.04370 0.09370 0.23750   
has_ev            1537672 0.05587 0.22967    0       0       0       0   

                       max  
consumption       13.22500  
generation         7.71250  
total_consumption 13.22500  
has_ev                   1  

df_consumption_app01.head()
                                 consumption  generation  total_consumption  \
meter time                                                                    
396   2021-11-01 11:00:00+11:00      0.04370           0            0.04370   
      2021-11-01 11:30:00+11:00      0.07500           0            0.07500   
      2021-11-01 12:00:00+11:00      0.11870           0            0.11870   
      2021-11-01 12:30:00+11:00      0.05620           0            0.05620   
      2021-11-01 13:00:00+11:00      0.04370           0            0.04370   

                                make model charger drive_kms  has_ev  
meter time                                                            
396   2021-11-01 11:00:00+11:00  NaN   NaN     NaN       NaN       0  
      2021-11-01 11:30:00+11:00  NaN   NaN     NaN       NaN       0  
      2021-11-01 12:00:00+11:00  NaN   NaN     NaN       NaN       0  
      2021-11-01 12:30:00+11:00  NaN   NaN     NaN       NaN       0  
      2021-11-01 13:00:00+11:00  NaN   NaN     NaN       NaN       0

In [43]:
df_power_quality_app01 = df_power_quality.copy()
df_power_quality_app01.info()
display('df_power_quality_app01.describe().T', f'df_power_quality_app01.head()')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10416147 entries, (396, Timestamp('2021-11-01 00:00:00+1100', tz='dateutil/Australia/Melbourne')) to (792189, Timestamp('2021-11-29 23:55:00+1100', tz='dateutil/Australia/Melbourne'))
Data columns (total 11 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   channel_cd                 category
 1   voltage_lvt                float64 
 2   amps                       float64 
 3   power_factor_pf            float64 
 4   number_of_phases           int64   
 5   overground_or_underground  category
 6   make                       object  
 7   model                      object  
 8   charger                    object  
 9   drive_kms                  object  
 10  has_ev                     int32   
dtypes: category(2), float64(3), int32(1), int64(1), object(4)
memory usage: 757.5+ MB


df_power_quality_app01.describe().T
                    count      mean      std       min       25%       50%  \
voltage_lvt       9222095 237.56618 23.23659         0 236.27000 240.14000   
amps             10416147   1.25739  3.62305 -57.64285   0.03659   0.70525   
power_factor_pf  10416147  -0.01402  0.83283        -1  -0.82900  -0.31800   
number_of_phases 10416147   1.33285  0.74492         1         1         1   
has_ev           10416147   0.07806  0.26827         0         0         0   

                       75%       max  
voltage_lvt      243.25000 259.99000  
amps               1.46181  73.12625  
power_factor_pf    0.99700         1  
number_of_phases         1         3  
has_ev                   0         1  

df_power_quality_app01.head()
                                channel_cd  voltage_lvt    amps  \
meter time                                                        
396   2021-11-01 00:00:00+11:00          a    242.47000 0.34400   
      2021-11-01 00:05:00+11:00          a    244.95000 0.33518   
      2021-11-01 00:10:00+11:00          a    243.59000 0.33152   
      2021-11-01 00:15:00+11:00          a    244.58000 0.56629   
      2021-11-01 00:20:00+11:00          a    245.43000 0.54029   

                                 power_factor_pf  number_of_phases  \
meter time                                                           
396   2021-11-01 00:00:00+11:00                1                 1   
      2021-11-01 00:05:00+11:00         -0.54300                 1   
      2021-11-01 00:10:00+11:00         -0.55200                 1   
      2021-11-01 00:15:00+11:00         -0.97300                 1   
      2021-11-01 00:20:00+11:00         -0.96800                 1   

                                overground_or_underground make model charger  \
meter time                                                                     
396   2021-11-01 00:00:00+11:00                        OH  NaN   NaN     NaN   
      2021-11-01 00:05:00+11:00                        OH  NaN   NaN     NaN   
      2021-11-01 00:10:00+11:00                        OH  NaN   NaN     NaN   
      2021-11-01 00:15:00+11:00                        OH  NaN   NaN     NaN   
      2021-11-01 00:20:00+11:00                        OH  NaN   NaN     NaN   

                                drive_kms  has_ev  
meter time                                         
396   2021-11-01 00:00:00+11:00       NaN       0  
      2021-11-01 00:05:00+11:00       NaN       0  
      2021-11-01 00:10:00+11:00       NaN       0  
      2021-11-01 00:15:00+11:00       NaN       0  
      2021-11-01 00:20:00+11:00       NaN       0

In [46]:
df_power_quality_app01.query("number_of_phases == 3 | channel_cd == 'a'")\
    .reset_index().groupby(['meter', 'time'])\
        .aggregate({
            'amps': 'sum',
            'voltage_lvt': 'mean',
            'power_factor_pf': 'mean',
            'number_of_phases': 'first',
            
        })

amps  voltage_lvt  power_factor_pf  \
meter  time                                                              
396    2021-11-01 00:00:00+11:00 0.34400    242.47000                1   
       2021-11-01 00:05:00+11:00 0.33518    244.95000         -0.54300   
       2021-11-01 00:10:00+11:00 0.33152    243.59000         -0.55200   
       2021-11-01 00:15:00+11:00 0.56629    244.58000         -0.97300   
       2021-11-01 00:20:00+11:00 0.54029    245.43000         -0.96800   
...                                  ...          ...              ...   
792189 2021-11-29 23:35:00+11:00       0    238.60000                1   
       2021-11-29 23:40:00+11:00       0    239.40000                1   
       2021-11-29 23:45:00+11:00       0    240.70000                1   
       2021-11-29 23:50:00+11:00       0    240.70000                1   
       2021-11-29 23:55:00+11:00       0    239.80000                1   

                                  number_of_phases  
meter  time                                         
396    2021-11-01 00:00:00+11:00                 1  
       2021-11-01 00:05:00+11:00                 1  
       2021-11-01 00:10:00+11:00                 1  
       2021-11-01 00:15:00+11:00                 1  
       2021-11-01 00:20:00+11:00                 1  
...                                            ...  
792189 2021-11-29 23:35:00+11:00                 1  
       2021-11-29 23:40:00+11:00                 1  
       2021-11-29 23:45:00+11:00                 1  
       2021-11-29 23:50:00+11:00                 1  
       2021-11-29 23:55:00+11:00                 1  

[8067436 rows x 4 columns]

# Others

In [ ]:
df_merged_hanna = pd.read_csv(r'C:\Users\khalil\LocalDrive\ev_consume_quality.csv')
df_merged_hanna.head()